In [31]:
import org.apache.spark.sql.SparkSession

lastException: Throwable = null


In [32]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", "/user/itv001389/warehouse").
    enableHiveSupport.
    master("yarn").
    appName("Getting Started - Spark with Scala").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@37c670fc


org.apache.spark.sql.SparkSession@37c670fc

In [33]:
spark.sql("use raj")

[]

In [4]:
val df=spark.read.schema("order_item_id int,order_item_order_id int,order_item_product_id int,order_item_quantity int,order_item_subtotal decimal(17,4),order_item_product_price decimal(17,4)").csv("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/order_item_new")

df = [order_item_id: int, order_item_order_id: int ... 4 more fields]


[order_item_id: int, order_item_order_id: int ... 4 more fields]

In [10]:
df.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|           299.9800|                299.9800|
|            2|                  2|                 1073|                  1|           199.9900|                199.9900|
|            3|                  2|                  502|                  5|           250.0000|                 50.0000|
|            4|                  2|                  403|                  1|           129.9900|                129.9900|
|            5|                  4|                  897|                  2|            49.9800|                 24.9900|
|            6| 

In [5]:
df.createOrReplaceTempView("orderItem")

In [12]:
spark.sql("select * from orderItem limit 10").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|           299.9800|                299.9800|
|            2|                  2|                 1073|                  1|           199.9900|                199.9900|
|            3|                  2|                  502|                  5|           250.0000|                 50.0000|
|            4|                  2|                  403|                  1|           129.9900|                129.9900|
|            5|                  4|                  897|                  2|            49.9800|                 24.9900|
|            6| 

In [17]:
spark.sql("select order_item_id,order_item_order_id,order_item_subtotal,sum(order_item_subtotal) over (partition by order_item_order_id) rev from orderItem order by order_item_order_id,order_item_id").show()

+-------------+-------------------+-------------------+---------+
|order_item_id|order_item_order_id|order_item_subtotal|      rev|
+-------------+-------------------+-------------------+---------+
|            1|                  1|           299.9800| 299.9800|
|            2|                  2|           199.9900| 579.9800|
|            3|                  2|           250.0000| 579.9800|
|            4|                  2|           129.9900| 579.9800|
|            5|                  4|            49.9800| 699.8500|
|            6|                  4|           299.9500| 699.8500|
|            7|                  4|           150.0000| 699.8500|
|            8|                  4|           199.9200| 699.8500|
|            9|                  5|           299.9800|1129.8600|
|           10|                  5|           299.9500|1129.8600|
|           11|                  5|            99.9600|1129.8600|
|           12|                  5|           299.9800|1129.8600|
|         

In [6]:
spark.conf.set("spark.sql.shuffle.partitions","2")

In [7]:
val df1=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/orders/part-00000-6502fdf1-89f6-48b4-ae4c-6c0376ab98ce-c000.snappy.parquet")

df1 = [order_id: int, Order_date: timestamp ... 2 more fields]


[order_id: int, Order_date: timestamp ... 2 more fields]

In [8]:
df1.createOrReplaceTempView("order")

In [21]:
spark.sql("select * from order limit 10").show()

+--------+-------------------+----------------+---------------+
|order_id|         Order_date|Order_product_id|   Order_status|
+--------+-------------------+----------------+---------------+
|       1|2013-07-25 00:00:00|           11599|         CLOSED|
|       2|2013-07-25 00:00:00|             256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|           12111|       COMPLETE|
|       4|2013-07-25 00:00:00|            8827|         CLOSED|
|       5|2013-07-25 00:00:00|           11318|       COMPLETE|
|       6|2013-07-25 00:00:00|            7130|       COMPLETE|
|       7|2013-07-25 00:00:00|            4530|       COMPLETE|
|       8|2013-07-25 00:00:00|            2911|     PROCESSING|
|       9|2013-07-25 00:00:00|            5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|            5648|PENDING_PAYMENT|
+--------+-------------------+----------------+---------------+



In [26]:
spark.sql("select o.Order_date,oi.order_item_product_id,sum(oi.order_item_subtotal) rev from order o join orderItem oi on o.Order_id=oi.order_item_order_id where Order_status in ('COMPLETE','CLOSED') group by Order_date,order_item_product_id order by Order_date,order_item_product_id").show()

+-------------------+---------------------+---------+
|         Order_date|order_item_product_id|      rev|
+-------------------+---------------------+---------+
|2013-07-25 00:00:00|                   24| 319.9600|
|2013-07-25 00:00:00|                   93|  74.9700|
|2013-07-25 00:00:00|                  134| 100.0000|
|2013-07-25 00:00:00|                  191|5099.4900|
|2013-07-25 00:00:00|                  226| 599.9900|
|2013-07-25 00:00:00|                  365|3359.4400|
|2013-07-25 00:00:00|                  403|1949.8500|
|2013-07-25 00:00:00|                  502|1650.0000|
|2013-07-25 00:00:00|                  572| 119.9700|
|2013-07-25 00:00:00|                  625| 199.9900|
|2013-07-25 00:00:00|                  627|1079.7300|
|2013-07-25 00:00:00|                  666| 109.9900|
|2013-07-25 00:00:00|                  705| 119.9900|
|2013-07-25 00:00:00|                  725| 108.0000|
|2013-07-25 00:00:00|                  810|  79.9600|
|2013-07-25 00:00:00|       

In [27]:
spark.sql("select o.Order_date,oi.order_item_product_id,sum(oi.order_item_subtotal) rev from order o join orderItem oi on o.Order_id=oi.order_item_order_id where Order_status in ('COMPLETE','CLOSED') group by Order_date,order_item_product_id having rev>=1000").show()

+-------------------+---------------------+----------+
|         Order_date|order_item_product_id|       rev|
+-------------------+---------------------+----------+
|2013-12-10 00:00:00|                  191| 4699.5300|
|2013-12-10 00:00:00|                  502| 2750.0000|
|2013-12-11 00:00:00|                 1004|14799.2600|
|2013-12-11 00:00:00|                  502| 2950.0000|
|2013-12-12 00:00:00|                  957| 4499.7000|
|2013-12-12 00:00:00|                 1004| 8399.5800|
|2013-12-12 00:00:00|                  191| 3299.6700|
|2013-12-12 00:00:00|                  403| 2339.8200|
|2013-12-12 00:00:00|                  365| 3479.4200|
|2013-12-12 00:00:00|                  627| 1159.7100|
|2013-12-13 00:00:00|                  502| 2950.0000|
|2013-12-13 00:00:00|                  365| 2879.5200|
|2013-12-14 00:00:00|                  365| 3119.4800|
|2013-12-14 00:00:00|                  502| 2050.0000|
|2013-12-14 00:00:00|                 1073| 2399.8800|
|2013-12-1

In [29]:
spark.sql("select * from (select o.Order_date,oi.order_item_product_id,sum(oi.order_item_subtotal) rev from order o join orderItem oi on o.Order_id=oi.order_item_order_id where Order_status in ('COMPLETE','CLOSED') group by Order_date,order_item_product_id) q where rev>=1000").show()

+-------------------+---------------------+----------+
|         Order_date|order_item_product_id|       rev|
+-------------------+---------------------+----------+
|2013-12-10 00:00:00|                  191| 4699.5300|
|2013-12-10 00:00:00|                  502| 2750.0000|
|2013-12-11 00:00:00|                 1004|14799.2600|
|2013-12-11 00:00:00|                  502| 2950.0000|
|2013-12-12 00:00:00|                  957| 4499.7000|
|2013-12-12 00:00:00|                 1004| 8399.5800|
|2013-12-12 00:00:00|                  191| 3299.6700|
|2013-12-12 00:00:00|                  403| 2339.8200|
|2013-12-12 00:00:00|                  365| 3479.4200|
|2013-12-12 00:00:00|                  627| 1159.7100|
|2013-12-13 00:00:00|                  502| 2950.0000|
|2013-12-13 00:00:00|                  365| 2879.5200|
|2013-12-14 00:00:00|                  365| 3119.4800|
|2013-12-14 00:00:00|                  502| 2050.0000|
|2013-12-14 00:00:00|                 1073| 2399.8800|
|2013-12-1

lastException: Throwable = null


In [38]:
spark.sql("select o.Order_date,oi.order_item_product_id,sum(oi.order_item_subtotal) rev from order o join orderItem oi on o.Order_id=oi.order_item_order_id where Order_status in ('COMPLETE','CLOSED') group by Order_date,order_item_product_id having sum(oi.order_item_subtotal)>=1000").show()

+-------------------+---------------------+---------+
|         Order_date|order_item_product_id|      rev|
+-------------------+---------------------+---------+
|2013-08-16 00:00:00|                  365|3719.3800|
|2013-09-21 00:00:00|                  403|2469.8100|
|2013-09-28 00:00:00|                 1014|3748.5000|
|2013-12-24 00:00:00|                  957|4799.6800|
|2014-02-25 00:00:00|                  957|8099.4600|
|2014-04-18 00:00:00|                 1073|5199.7400|
|2014-04-30 00:00:00|                  191|2199.7800|
|2014-05-25 00:00:00|                 1004|5199.7400|
|2014-06-24 00:00:00|                  957|2999.8000|
|2014-06-29 00:00:00|                  502|2650.0000|
|2013-07-29 00:00:00|                  502|5650.0000|
|2013-09-03 00:00:00|                  365|4379.2700|
|2013-09-08 00:00:00|                  502|5150.0000|
|2013-09-08 00:00:00|                  191|3999.6000|
|2013-10-17 00:00:00|                 1073|2799.8600|
|2013-11-11 00:00:00|       

In [34]:
val df2=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/emp_join")

df2 = [EMPNO: int, ENAME: string ... 6 more fields]


[EMPNO: int, ENAME: string ... 6 more fields]

In [40]:
df2.show()

+-----+------+---------+----+----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+

In [35]:
df2.createOrReplaceTempView("emp")

In [7]:
val df3=spark.read.parquet("hdfs://m01.itversity.com:9000/user/itv001389/warehouse/raj.db/dep_join")

df3 = [DEPTNO: int, DNAME: string ... 1 more field]


[DEPTNO: int, DNAME: string ... 1 more field]

In [11]:
df3.show()

+------+----------+--------+
|DEPTNO|     DNAME|     LOC|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [8]:
df3.createOrReplaceTempView("dept")

In [10]:
spark.sql("select DEPTNO,sum(SAL) from emp  group by DEPTNO").show()

+------+--------+
|DEPTNO|sum(SAL)|
+------+--------+
|    20|   10875|
|    10|    8750|
|    30|    9400|
+------+--------+



In [21]:
spark.sql("select EMPNO,DEPTNO,SAL,sum(SAL) over (partition by DEPTNO) total_sal from emp order by DEPTNO,SAL").show()

+-----+------+----+---------+
|EMPNO|DEPTNO| SAL|total_sal|
+-----+------+----+---------+
| 7934|    10|1300|     8750|
| 7782|    10|2450|     8750|
| 7839|    10|5000|     8750|
| 7369|    20| 800|    10875|
| 7876|    20|1100|    10875|
| 7566|    20|2975|    10875|
| 7902|    20|3000|    10875|
| 7788|    20|3000|    10875|
| 7900|    30| 950|     9400|
| 7521|    30|1250|     9400|
| 7654|    30|1250|     9400|
| 7844|    30|1500|     9400|
| 7499|    30|1600|     9400|
| 7698|    30|2850|     9400|
+-----+------+----+---------+



lastException: Throwable = null


In [26]:
spark.sql("select * from (select EMPNO,DEPTNO,SAL,avg(SAL) over (partition by DEPTNO) avg_sal from emp ) q where q.SAL>=avg_sal order by DEPTNO,SAL").show()

+-----+------+----+------------------+
|EMPNO|DEPTNO| SAL|           avg_sal|
+-----+------+----+------------------+
| 7839|    10|5000|2916.6666666666665|
| 7566|    20|2975|            2175.0|
| 7788|    20|3000|            2175.0|
| 7902|    20|3000|            2175.0|
| 7499|    30|1600|1566.6666666666667|
| 7698|    30|2850|1566.6666666666667|
+-----+------+----+------------------+



In [40]:
spark.sql("select EMPNO,DEPTNO,SAL,lead(EMPNO) over (order by SAL DESC) lead_salary from emp").show()

+-----+------+----+-----------+
|EMPNO|DEPTNO| SAL|lead_salary|
+-----+------+----+-----------+
| 7839|    10|5000|       7788|
| 7788|    20|3000|       7902|
| 7902|    20|3000|       7566|
| 7566|    20|2975|       7698|
| 7698|    30|2850|       7782|
| 7782|    10|2450|       7499|
| 7499|    30|1600|       7844|
| 7844|    30|1500|       7934|
| 7934|    10|1300|       7521|
| 7521|    30|1250|       7654|
| 7654|    30|1250|       7876|
| 7876|    20|1100|       7900|
| 7900|    30| 950|       7369|
| 7369|    20| 800|       null|
+-----+------+----+-----------+



In [12]:
spark.sql("select EMPNO,DEPTNO,SAL,lead(EMPNO) over (order by SAL DESC) Lead_emp_id,lead(SAL) over (order by SAL DESC) lead_salary from emp order by SAL DESC").show()

+-----+------+----+-----------+-----------+
|EMPNO|DEPTNO| SAL|Lead_emp_id|lead_salary|
+-----+------+----+-----------+-----------+
| 7839|    10|5000|       7788|       3000|
| 7788|    20|3000|       7902|       3000|
| 7902|    20|3000|       7566|       2975|
| 7566|    20|2975|       7698|       2850|
| 7698|    30|2850|       7782|       2450|
| 7782|    10|2450|       7499|       1600|
| 7499|    30|1600|       7844|       1500|
| 7844|    30|1500|       7934|       1300|
| 7934|    10|1300|       7521|       1250|
| 7654|    30|1250|       7876|       1100|
| 7521|    30|1250|       7654|       1250|
| 7876|    20|1100|       7900|        950|
| 7900|    30| 950|       7369|        800|
| 7369|    20| 800|       null|       null|
+-----+------+----+-----------+-----------+



In [13]:
spark.sql("select EMPNO,DEPTNO,SAL,lead(EMPNO) over (partition by DEPTNO order by SAL DESC) Lead_emp_id,lead(SAL) over (order by SAL DESC) lead_salary from emp order by SAL DESC").show()

+-----+------+----+-----------+-----------+
|EMPNO|DEPTNO| SAL|Lead_emp_id|lead_salary|
+-----+------+----+-----------+-----------+
| 7839|    10|5000|       7782|       3000|
| 7788|    20|3000|       7902|       3000|
| 7902|    20|3000|       7566|       2975|
| 7566|    20|2975|       7876|       2850|
| 7698|    30|2850|       7499|       2450|
| 7782|    10|2450|       7934|       1600|
| 7499|    30|1600|       7844|       1500|
| 7844|    30|1500|       7521|       1300|
| 7934|    10|1300|       null|       1250|
| 7654|    30|1250|       7900|       1100|
| 7521|    30|1250|       7654|       1250|
| 7876|    20|1100|       7369|        950|
| 7900|    30| 950|       null|        800|
| 7369|    20| 800|       null|       null|
+-----+------+----+-----------+-----------+



In [15]:
spark.sql("select EMPNO,DEPTNO,SAL,first_value(ENAME) over (partition by DEPTNO order by SAL DESC) EMP_NAME from emp order by DEPTNO,SAL DESC").show()

+-----+------+----+--------+
|EMPNO|DEPTNO| SAL|EMP_NAME|
+-----+------+----+--------+
| 7839|    10|5000|    KING|
| 7782|    10|2450|    KING|
| 7934|    10|1300|    KING|
| 7902|    20|3000|   SCOTT|
| 7788|    20|3000|   SCOTT|
| 7566|    20|2975|   SCOTT|
| 7876|    20|1100|   SCOTT|
| 7369|    20| 800|   SCOTT|
| 7698|    30|2850|   BLAKE|
| 7499|    30|1600|   BLAKE|
| 7844|    30|1500|   BLAKE|
| 7654|    30|1250|   BLAKE|
| 7521|    30|1250|   BLAKE|
| 7900|    30| 950|   BLAKE|
+-----+------+----+--------+



In [22]:
spark.sql("select EMPNO,DEPTNO,SAL,min(SAL) over (partition by DEPTNO) MIN_SALARY from emp order by DEPTNO,SAL DESC").show()

+-----+------+----+----------+
|EMPNO|DEPTNO| SAL|MIN_SALARY|
+-----+------+----+----------+
| 7839|    10|5000|      1300|
| 7782|    10|2450|      1300|
| 7934|    10|1300|      1300|
| 7902|    20|3000|       800|
| 7788|    20|3000|       800|
| 7566|    20|2975|       800|
| 7876|    20|1100|       800|
| 7369|    20| 800|       800|
| 7698|    30|2850|       950|
| 7499|    30|1600|       950|
| 7844|    30|1500|       950|
| 7654|    30|1250|       950|
| 7521|    30|1250|       950|
| 7900|    30| 950|       950|
+-----+------+----+----------+



lastException: Throwable = null


In [23]:
spark.sql("select EMPNO,DEPTNO,SAL,last_value(SAL) over (partition by DEPTNO order by SAL DESC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) last_salary from emp order by DEPTNO,SAL DESC").show()

+-----+------+----+-----------+
|EMPNO|DEPTNO| SAL|last_salary|
+-----+------+----+-----------+
| 7839|    10|5000|       1300|
| 7782|    10|2450|       1300|
| 7934|    10|1300|       1300|
| 7788|    20|3000|        800|
| 7902|    20|3000|        800|
| 7566|    20|2975|        800|
| 7876|    20|1100|        800|
| 7369|    20| 800|        800|
| 7698|    30|2850|        950|
| 7499|    30|1600|        950|
| 7844|    30|1500|        950|
| 7654|    30|1250|        950|
| 7521|    30|1250|        950|
| 7900|    30| 950|        950|
+-----+------+----+-----------+



In [24]:
spark.sql("select EMPNO,DEPTNO,SAL,max(SAL) over (partition by DEPTNO) MAX_SALARY from emp order by DEPTNO,SAL DESC").show()

+-----+------+----+----------+
|EMPNO|DEPTNO| SAL|MAX_SALARY|
+-----+------+----+----------+
| 7839|    10|5000|      5000|
| 7782|    10|2450|      5000|
| 7934|    10|1300|      5000|
| 7788|    20|3000|      3000|
| 7902|    20|3000|      3000|
| 7566|    20|2975|      3000|
| 7876|    20|1100|      3000|
| 7369|    20| 800|      3000|
| 7698|    30|2850|      2850|
| 7499|    30|1600|      2850|
| 7844|    30|1500|      2850|
| 7654|    30|1250|      2850|
| 7521|    30|1250|      2850|
| 7900|    30| 950|      2850|
+-----+------+----+----------+



In [25]:
spark.sql("select EMPNO,DEPTNO,SAL,rank() over (partition by DEPTNO order by SAL DESC) RANK from emp order by deptno,sal desc").show()

+-----+------+----+----+
|EMPNO|DEPTNO| SAL|RANK|
+-----+------+----+----+
| 7839|    10|5000|   1|
| 7782|    10|2450|   2|
| 7934|    10|1300|   3|
| 7788|    20|3000|   1|
| 7902|    20|3000|   1|
| 7566|    20|2975|   3|
| 7876|    20|1100|   4|
| 7369|    20| 800|   5|
| 7698|    30|2850|   1|
| 7499|    30|1600|   2|
| 7844|    30|1500|   3|
| 7654|    30|1250|   4|
| 7521|    30|1250|   4|
| 7900|    30| 950|   6|
+-----+------+----+----+



In [28]:
spark.sql("select EMPNO,DEPTNO,SAL,dense_rank() over (partition by DEPTNO order by SAL DESC) DENSE_RANK from emp order by deptno,sal desc").show()

+-----+------+----+----------+
|EMPNO|DEPTNO| SAL|DENSE_RANK|
+-----+------+----+----------+
| 7839|    10|5000|         1|
| 7782|    10|2450|         2|
| 7934|    10|1300|         3|
| 7788|    20|3000|         1|
| 7902|    20|3000|         1|
| 7566|    20|2975|         2|
| 7876|    20|1100|         3|
| 7369|    20| 800|         4|
| 7698|    30|2850|         1|
| 7499|    30|1600|         2|
| 7844|    30|1500|         3|
| 7654|    30|1250|         4|
| 7521|    30|1250|         4|
| 7900|    30| 950|         5|
+-----+------+----+----------+



lastException: Throwable = null


In [29]:
spark.sql("select EMPNO,DEPTNO,SAL,row_number() over (partition by DEPTNO order by SAL DESC) RANK from emp order by deptno,sal desc").show()

+-----+------+----+----+
|EMPNO|DEPTNO| SAL|RANK|
+-----+------+----+----+
| 7839|    10|5000|   1|
| 7782|    10|2450|   2|
| 7934|    10|1300|   3|
| 7788|    20|3000|   1|
| 7902|    20|3000|   2|
| 7566|    20|2975|   3|
| 7876|    20|1100|   4|
| 7369|    20| 800|   5|
| 7698|    30|2850|   1|
| 7499|    30|1600|   2|
| 7844|    30|1500|   3|
| 7654|    30|1250|   5|
| 7521|    30|1250|   4|
| 7900|    30| 950|   6|
+-----+------+----+----+



In [36]:
spark.sql("select EMPNO,DEPTNO,SAL,rank() over (partition by DEPTNO order by SAL DESC) RANK,dense_rank() over (partition by DEPTNO order by SAL DESC) DENSE_RANK,row_number() over (partition by DEPTNO order by SAL DESC) ROW_NUM from emp order by deptno,sal desc").show()

+-----+------+----+----+----------+-------+
|EMPNO|DEPTNO| SAL|RANK|DENSE_RANK|ROW_NUM|
+-----+------+----+----+----------+-------+
| 7839|    10|5000|   1|         1|      1|
| 7782|    10|2450|   2|         2|      2|
| 7934|    10|1300|   3|         3|      3|
| 7788|    20|3000|   1|         1|      1|
| 7902|    20|3000|   1|         1|      2|
| 7566|    20|2975|   3|         2|      3|
| 7876|    20|1100|   4|         3|      4|
| 7369|    20| 800|   5|         4|      5|
| 7698|    30|2850|   1|         1|      1|
| 7499|    30|1600|   2|         2|      2|
| 7844|    30|1500|   3|         3|      3|
| 7654|    30|1250|   4|         4|      5|
| 7521|    30|1250|   4|         4|      4|
| 7900|    30| 950|   6|         5|      6|
+-----+------+----+----+----------+-------+

